In [8]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install python-dotenv mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import mysql.connector
from mysql.connector import errorcode

import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
# Crea la base de datos 'HR'
cnx = mysql.connector.connect(
    user='root',
    password='AlumnaAdalab',
    host='127.0.0.1',     
)
cur = cnx.cursor()

try:
    cur.execute("CREATE DATABASE HR")
    print("Base de datos 'HR' creada exitosamente.")
except mysql.connector.Error as err:
    print(err)
    print("Código de Error:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Mensaje", err.msg)

cnx.close()
# Si ejecutas esta celda dos veces, obtendrás un aviso de que la base de datos ya existe.


1007 (HY000): Can't create database 'HR'; database exists
Código de Error: 1007
SQLSTATE HY000
Mensaje Can't create database 'HR'; database exists


In [4]:
try:
    # Intenta hacer la conexión con la base de datos 'HR'
    cnx = mysql.connector.connect(
        user='root',
        password='AlumnaAdalab',
        host='127.0.0.1',      # o la IP del servidor MySQL
        database='HR'
    )
    cur = cnx.cursor()
    
    cur.execute("""
    CREATE TABLE IF NOT EXISTS employees (
        employeenumber INTEGER PRIMARY KEY,
        gender VARCHAR(10),
        maritalstatus VARCHAR(20),
        attrition VARCHAR(10),
        datebirth INTEGER
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS satisfaction (
        employeenumber INTEGER,
        environmentsatisfaction INTEGER,
        jobsatisfaction INTEGER,
        relationshipsatisfaction INTEGER,
        jobinvolvement INTEGER,
        worklifebalance INTEGER,
        performancerating INTEGER,
        CONSTRAINT fk_satisfaction_emp FOREIGN KEY (employeenumber) REFERENCES employees(employeenumber)
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS work (
        employeenumber INTEGER,
        department VARCHAR(75),
        jobrole VARCHAR(75),
        joblevel INTEGER,
        remotework VARCHAR(10),
        standardhours VARCHAR(20),
        CONSTRAINT fk_work_emp FOREIGN KEY (employeenumber) REFERENCES employees(employeenumber)
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS history (
        employeenumber INTEGER,
        totalworkingyears FLOAT,
        yearsatcompany INTEGER,
        yearswithcurrmanager INTEGER,
        yearssincelastpromotion INTEGER,
        trainingtimeslastyear INTEGER,        
        CONSTRAINT fk_history_emp FOREIGN KEY (employeenumber) REFERENCES employees(employeenumber)
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS conditions (
        employeenumber INTEGER,
        monthlyincome FLOAT,
        monthlyrate FLOAT,
        dailyrate FLOAT,
        hourlyrate FLOAT,
        salary FLOAT,
        stockoptionlevel INTEGER,
        percentsalaryhike INTEGER,
        overtime VARCHAR(25),
        CONSTRAINT fk_conditions_emp FOREIGN KEY (employeenumber) REFERENCES employees(employeenumber)
    );
    """)

except mysql.connector.Error as err:
    # Si es un error de acceso denegado (ej. contraseña o usuario incorrecto)
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Algo está mal con tu nombre de usuario o contraseña.") #
    # Si la base de datos no existe
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe.") #
    # Para cualquier otro tipo de error
    else:
        print(err) #
        print("Código de Error:", err.errno) #
        print("SQLSTATE", err.sqlstate) #
        print("Mensaje", err.msg) #
else:
    # Si la conexión fue exitosa, cierra la conexión
    cnx.close()
    print("Conexión cerrada.")

Conexión cerrada.


In [5]:
import pandas as pd
import mysql.connector

In [6]:
cnx = mysql.connector.connect(
    user='root',
    password='AlumnaAdalab',
    host='127.0.0.1',
    database='HR'
)

mycursor = cnx.cursor() # Iniciamos el cursor

mycursor.execute("SHOW TABLES") # Ejecutamos la query

# El cursor se convierte en un iterable al que podemos acceder, por ejemplo, con un bucle for
print("Tablas en la base de datos 'HR':")
for x in mycursor:
    print(x) # Devuelve cada resultado como una tupla

cnx.close()

Tablas en la base de datos 'HR':
('conditions',)
('employees',)
('history',)
('satisfaction',)
('work',)


In [8]:
# Cargar el CSV
ruta_csv = 'hr_clean_data_v6_BBDD.csv'  

# Leer archivo CSV usando pandas y guardar los datos en un DF
df = pd.read_csv(ruta_csv)

try:
    # Conectar a la base de datos MySQL con usuario, contraseña, host y base de datos especificados
    cnx = mysql.connector.connect(
        user='root',
        password='AlumnaAdalab',
        host='127.0.0.1',
        database='HR'
    )
    # Crear un cursor para ejecutar comandos SQL
    cur = cnx.cursor()

    # Iterar sobre cada fila del df para insertar datos en la tabla employees
    for _, row in df.iterrows(): # _ lo estamos usando como variable temporal; lo usamos porque no nos importa el número de iteraciones
                                 # es como usar un i (tipo "for i in…"), pero aquí dejamos claro que no nos sirve después para nada dentro del bucle
        cur.execute("""
        INSERT INTO employees (employeenumber, gender, maritalstatus, attrition, datebirth)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE employeenumber=employeenumber
        """, (
            int(row['employeenumber']),  # Convertir a entero para el número de empleado
            row['gender'],              
            row['maritalstatus'],       
            row['attrition'],           
            int(row['datebirth'])       # Fecha de nacimiento como entero
        ))

    # Insertar datos en la tabla satisfaction para cada fila de df
    for _, row in df.iterrows():
        cur.execute("""
        INSERT INTO satisfaction (employeenumber, environmentsatisfaction, jobsatisfaction, relationshipsatisfaction,
                                  jobinvolvement, worklifebalance, performancerating)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE employeenumber=employeenumber
        """, (
            row['employeenumber'],          
            row['environmentsatisfaction'],
            row['jobsatisfaction'],         
            row['relationshipsatisfaction'],
            row['jobinvolvement'],       
            row['worklifebalance'],        
            row['performancerating']       
        ))

    # Insertar datos en la tabla work para cada fila de df
    for _, row in df.iterrows():
        cur.execute("""
        INSERT INTO work (employeenumber, department, jobrole, joblevel, remotework, standardhours)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE employeenumber=employeenumber
        """, (
            row['employeenumber'],   
            row['department'],       
            row['jobrole'],          
            row['joblevel'],         
            row['remotework'],       
            row['standardhours']     
        ))

    # Insertar datos en la tabla history para cada fila de df
    for _, row in df.iterrows():
        cur.execute("""
        INSERT INTO history (employeenumber, totalworkingyears, yearsatcompany, yearswithcurrmanager,
                             yearssincelastpromotion, trainingtimeslastyear)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE employeenumber=employeenumber
        """, (
            row['employeenumber'],          
            row['totalworkingyears'],       
            row['yearsatcompany'],          
            row['yearswithcurrmanager'],   
            row['yearssincelastpromotion'],
            row['trainingtimeslastyear']   
        ))

    # Insertar datos en la tabla conditions para cada fila de df
    for _, row in df.iterrows():
        cur.execute("""
        INSERT INTO conditions (employeenumber, monthlyincome, monthlyrate, dailyrate, hourlyrate, salary,
                                stockoptionlevel, percentsalaryhike, overtime)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE employeenumber=employeenumber
        """, (
            row['employeenumber'],       
            row['monthlyincome'],        
            row['monthlyrate'],          
            row['dailyrate'],            
            row['hourlyrate'],           
            row['salary'],              
            row['stockoptionlevel'],     
            row['percentsalaryhike'],   
            row['overtime']              
        ))

    # Confirmar todos los cambios en la base de datos
    cnx.commit()

# Manejar errores comunes al conectar o ejecutar queries en MySQL
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Algo está mal con tu nombre de usuario o contraseña.")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe.")
    else:
        print(err)

# Asegurar cierre del cursor y conexión aunque haya errores
finally:
    cur.close()
    cnx.close()

# Informar que los datos se han insertado correctamente
print("Datos insertados correctamente en las tablas")


Datos insertados correctamente en las tablas


In [10]:
# Verificación de inserciones

try:
    # Conectar a la base de datos MySQL
    cnx = mysql.connector.connect(
        user='root',
        password='AlumnaAdalab',
        host='127.0.0.1',
        database='HR'
    )
    cur = cnx.cursor()

    # Consultar algunas filas de la tabla employees
    cur.execute("SELECT * FROM employees LIMIT 5")
    filas_employees = cur.fetchall()
    print("Datos en tabla employees:")
    for fila in filas_employees:
        print(fila)

    # Consultar algunas filas de la tabla satisfaction
    cur.execute("SELECT * FROM satisfaction LIMIT 5")
    filas_satisfaction = cur.fetchall()
    print("\nDatos en tabla satisfaction:")
    for fila in filas_satisfaction:
        print(fila)

    # Consultar algunas filas de la tabla work
    cur.execute("SELECT * FROM work LIMIT 5")
    filas_work = cur.fetchall()
    print("\nDatos en tabla work:")
    for fila in filas_work:
        print(fila)

    # Consultar algunas filas de la tabla history
    cur.execute("SELECT * FROM history LIMIT 5")
    filas_history = cur.fetchall()
    print("\nDatos en tabla history:")
    for fila in filas_history:
        print(fila)

    # Consultar algunas filas de la tabla conditions
    cur.execute("SELECT * FROM conditions LIMIT 5")
    filas_conditions = cur.fetchall()
    print("\nDatos en tabla conditions:")
    for fila in filas_conditions:
        print(fila)

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Error en usuario o contraseña.")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe.")
    else:
        print(err)
finally:
    cur.close()
    cnx.close()


Datos en tabla employees:
(1, 'female', 'desconocido', 'no', 1972)
(2, 'female', 'desconocido', 'no', 1971)
(3, 'female', 'married', 'no', 1981)
(4, 'male', 'married', 'no', 1976)
(5, 'male', 'divorced', 'no', 1977)

Datos en tabla satisfaction:
(1, 1, 3, 3, 3, 3, 3)
(2, 3, 3, 1, 2, 3, 3)
(3, 3, 4, 4, 3, 3, 3)
(4, 1, 3, 2, 3, 3, 3)
(5, 1, 1, 4, 4, 3, 3)

Datos en tabla work:
(1, 'desconocido', 'research_director', 5, 'yes', 'full_time')
(2, 'desconocido', 'manager', 5, 'yes', 'desconocido')
(3, 'research_&_development', 'manager', 5, 'yes', 'desconocido')
(4, 'desconocido', 'research_director', 4, 'no', 'full_time')
(5, 'desconocido', 'sales_executive', 4, 'no', 'desconocido')

Datos en tabla history:
(1, 10.0, 20, 15, 15, 5)
(2, 34.0, 33, 9, 11, 5)
(3, 22.0, 22, 15, 11, 3)
(4, 10.0, 20, 6, 5, 2)
(5, 10.0, 19, 8, 2, 5)

Datos en tabla conditions:
(1, 16280.8, 42330.2, 2015.72, 69.5321, 195370.0, 0, 13, 'no')
(2, 4492.84, 43331.2, 2063.39, 69.5321, 199990.0, 1, 14, 'desconocido')
(3, 44